<a href="https://colab.research.google.com/github/jPruim/cs344/blob/master/Project/LinAlg02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Proposal:
Use neural networks to try and solve common and less common linear algebra problems. This will involve using neural nets and CNN's to try and see how few nodes are necessary to solve some problems.

Specifically I hope to investigate determinants, eigenvalues, and inverses for matrices. Also, I hope to be able to attempt to do a decomposition. Part of the project is determining whether or not it is possible to do some of these approximations.

In [0]:
import numpy as np
from keras import layers
from keras import models
from keras.layers.advanced_activations import LeakyReLU
import numpy.linalg as linalg

In [45]:
#create datasets, normalized arrays of n^2 size.
#Thus all values are between -1 and 1
#No third set needed as the sets are random and different each time anyway
n = 7
train_set_size = 40000
train_set=np.random.random(size=(train_set_size, n, n))*2-1

val_set_size = 10000
val_set=np.random.random(size=(val_set_size, n, n))*2-1

#print shape
print(train_set[:1])
print(len(train_set))



#Shaped for simple neural networks
train_set2 = np.empty((train_set_size,n**2))
for i in range(train_set_size):
  train_set2[i] = np.ravel(train_set[i])
train_set2.reshape(train_set_size,n**2,1)
print(train_set2.shape)

val_set2 = np.empty((val_set_size,n**2))
for i in range(val_set_size):
  val_set2[i] = np.ravel(val_set[i])
val_set2.reshape(val_set_size,n**2,1)
print(val_set2.shape)

#Shaped for CNN neural networks
train_set3 = train_set.reshape(train_set_size,n,n,1)
val_set3 = val_set.reshape(val_set_size,n,n,1)


[[[ 0.08100883 -0.09832539 -0.23903097  0.0704677  -0.29418192
    0.26060713 -0.46434745]
  [-0.32374422 -0.0757782   0.78429317  0.0351878   0.89129148
   -0.38159363 -0.80601021]
  [-0.91122261 -0.94162794 -0.97734409  0.00102184  0.15422242
   -0.79000496  0.15099852]
  [ 0.53181025  0.12584413  0.0493785   0.19446916  0.54556067
    0.6990849  -0.81894988]
  [-0.77077547 -0.37407564  0.46348771 -0.63890725  0.83017821
   -0.56079956  0.93245966]
  [ 0.64292497  0.6829567  -0.879071   -0.4623105  -0.23462356
   -0.2515304  -0.72694737]
  [-0.13067754 -0.70919966  0.31260706 -0.74580409  0.30999527
    0.53212448 -0.74992097]]]
40000
(40000, 49)
(10000, 49)


In [46]:
#generate determinant labels
determinant_set_size = train_set_size
determinant_set = np.empty((determinant_set_size,1))
determinant_abs_set  = np.empty((determinant_set_size,1))
for i in range(0,determinant_set_size): 
  determinant_set[i]=linalg.det(train_set[i])
  determinant_abs_set[i] = abs(determinant_set[i])

print(sum(determinant_abs_set) / len(determinant_set))
val_determinant_set_size = val_set_size
val_determinant_set = np.empty((val_set_size,1))
for i in range(0,val_determinant_set_size): 
  val_determinant_set[i]= linalg.det(val_set[i])

print(determinant_set.shape)

[0.92339563]
(40000, 1)


In [47]:
#generate eigenvalue labels
eigenvalue_set_size = train_set_size
eigenvalue_set = np.empty((eigenvalue_set_size,n))
for i in range(0,train_set_size):
  eigenValues= linalg.eigvals(train_set[i])
  eigenvalue_set[i]= eigenValues

val_eigenvalue_set_size = val_set_size
val_eigenvalue_set = np.empty((val_eigenvalue_set_size,n))
for i in range(0,val_eigenvalue_set_size): 
  eigenValues = linalg.eigvals(train_set[i])
  val_eigenvalue_set[i]= eigenValues

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: ComplexWarning: Casting complex values to real discards the imaginary part
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: ComplexWarning: Casting complex values to real discards the imaginary part
  if sys.path[0] == '':


In [49]:
#start modeling.
#determinant modeling 1
detmodel = models.Sequential()

detmodel.add(layers.Dense(n**2, activation='relu',input_dim=n**2))
detmodel.add(layers.Dense(50, activation='relu'))
detmodel.add(layers.Dense(50, activation='relu'))
detmodel.add(layers.Dense(50, activation='relu'))
detmodel.add(layers.Dense(50, activation='relu'))
detmodel.add(layers.Dense(50, activation='relu'))
# model.add(layers.Dense(50, activation='relu'))
detmodel.add(layers.Dense(1, activation='relu'))
detmodel.compile(optimizer='adam',
              loss='mean_squared_error')
detmodel.summary()


Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_84 (Dense)             (None, 49)                2450      
_________________________________________________________________
dense_85 (Dense)             (None, 50)                2500      
_________________________________________________________________
dense_86 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_87 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_88 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_89 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_90 (Dense)             (None, 1)               

In [50]:
#Simple NN model for determinant with Relu
detmodel.fit(train_set2, determinant_set, epochs=5, batch_size=100)
detmodel.evaluate(val_set2, val_determinant_set)
print(detmodel.predict(val_set2[:10]))
print(val_determinant_set[:10])

Epoch 1/5
40000/40000 [==============================] - 1s 22us/step - loss: 2.2294
Epoch 2/5
40000/40000 [==============================] - 1s 18us/step - loss: 2.2291
Epoch 3/5
40000/40000 [==============================] - 1s 18us/step - loss: 2.2291
Epoch 4/5
40000/40000 [==============================] - 1s 17us/step - loss: 2.2291
Epoch 5/5
10000/10000 [==============================] - 0s 20us/step
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[-0.52567272]
 [ 1.23726643]
 [ 2.63876382]
 [ 0.09930698]
 [-1.00736075]
 [ 0.07695806]
 [-0.10954239]
 [ 0.14228992]
 [-0.24531103]
 [-1.06054693]]


In [63]:
#start modeling.
#determinant modeling
detmodel2 = models.Sequential()

detmodel2.add(layers.Dense(n**2, activation=LeakyReLU(alpha=0.5),input_dim=n**2))
detmodel2.add(layers.Dense(n**3, activation=LeakyReLU(alpha=0.3)))
detmodel2.add(layers.Dense(n**3, activation=LeakyReLU(alpha=0.3)))
detmodel2.add(layers.Dense(n**2, activation=LeakyReLU(alpha=0.3)))
detmodel2.add(layers.Dense(n**2, activation=LeakyReLU(alpha=0.5)))
# detmodel2.add(layers.Dense(50, activation='relu'))
# detmodel2.add(layers.Dense(50, activation='relu'))
detmodel2.add(layers.Dense(1, activation=LeakyReLU(alpha=0.5)))
detmodel2.compile(optimizer='adam',
              loss='mean_squared_error')
detmodel2.summary()



Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_107 (Dense)            (None, 49)                2450      
_________________________________________________________________
dense_108 (Dense)            (None, 343)               17150     
_________________________________________________________________
dense_109 (Dense)            (None, 343)               117992    
_________________________________________________________________
dense_110 (Dense)            (None, 49)                16856     
_________________________________________________________________
dense_111 (Dense)            (None, 49)                2450      
_________________________________________________________________
dense_112 (Dense)            (None, 1)                 50        
Total params: 156,948
Trainable params: 156,948
Non-trainable params: 0
_______________________________________________

/usr/local/lib/python3.6/dist-packages/keras/activations.py:235: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


In [66]:
#Simple NN model for determinant with Leaky RelU
detmodel2.fit(train_set2, determinant_set, epochs=50, batch_size=1000)
detmodel2.evaluate(val_set2, val_determinant_set)

print("On Training data")
print(detmodel2.predict(val_set2[:5]))
print(val_determinant_set[:5])

print("On Validation data")
print(detmodel2.predict(val_set2[:5]))
print(val_determinant_set[:5])

Epoch 1/50
40000/40000 [==============================] - 1s 29us/step - loss: 1.4669
Epoch 2/50
40000/40000 [==============================] - 1s 28us/step - loss: 1.4113
Epoch 3/50
40000/40000 [==============================] - 1s 29us/step - loss: 1.3628
Epoch 4/50
40000/40000 [==============================] - 1s 28us/step - loss: 1.3238
Epoch 5/50
40000/40000 [==============================] - 1s 29us/step - loss: 1.2969
Epoch 6/50
40000/40000 [==============================] - 1s 29us/step - loss: 1.2826
Epoch 7/50
40000/40000 [==============================] - 1s 29us/step - loss: 1.2313
Epoch 8/50
40000/40000 [==============================] - 1s 28us/step - loss: 1.2328
Epoch 9/50
40000/40000 [==============================] - 1s 28us/step - loss: 1.1630
Epoch 10/50
40000/40000 [==============================] - 1s 29us/step - loss: 1.1800
Epoch 11/50
40000/40000 [==============================] - 1s 28us/step - loss: 1.1132
Epoch 12/50
40000/40000 [===========================

In [0]:
detCNNmodel = models.Sequential()
detCNNmodel.add(layers.Conv2D((n-2)**2, (3, 3), activation='relu', input_shape=(n, n,1)))
detCNNmodel.add(layers.MaxPooling2D((2, 2)))
# Add layers to flatten the 2D image and then do a 10-way classification.
detCNNmodel.add(layers.Flatten())
detCNNmodel.add(layers.Dense(n**2, activation='relu'))
detCNNmodel.add(layers.Dense(1, activation='relu'))
detCNNmodel.compile(optimizer='adam',
              loss='mean_squared_error')

detCNNmodel.summary()

In [0]:
#CNN model for determinant
detmodel2.fit(train_set3, determinant_set, epochs=8, batch_size=100)
detmodel2.evaluate(val_set3, val_determinant_set)

Epoch 1/8
40000/40000 [==============================] - 1s 32us/step - loss: 2.2634
Epoch 2/8
40000/40000 [==============================] - 1s 30us/step - loss: 2.2631
Epoch 3/8
40000/40000 [==============================] - 1s 30us/step - loss: 2.2631
Epoch 4/8
40000/40000 [==============================] - 1s 29us/step - loss: 2.2631
Epoch 5/8
40000/40000 [==============================] - 1s 30us/step - loss: 2.2631
Epoch 6/8
40000/40000 [==============================] - 1s 29us/step - loss: 2.2631
Epoch 7/8
40000/40000 [==============================] - 1s 30us/step - loss: 2.2631
Epoch 8/8
10000/10000 [==============================] - 0s 27us/step


2.3721099333763123

In [0]:
#Eigenvalue modeling
print(eigenvalue_set.shape)

eigenmodel = models.Sequential()

# # Configure a convnet with 3 layers of convolutions and max pooling.
# model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.Flatten())

# Add layers to flatten the 2D image and then do a 10-way classification.

eigenmodel.add(layers.Dense(n**2, activation='relu',input_dim=n**2))
eigenmodel.add(layers.Dense(50, activation='relu',input_dim=n**2))
eigenmodel.add(layers.Dense(50, activation='relu',input_dim=n**2))
eigenmodel.add(layers.Dense(n, activation='relu'))
eigenmodel.compile(optimizer='adam',
              loss='mean_squared_error')
eigenmodel.summary()
eigenmodel.fit(train_set2, eigenvalue_set, epochs=20, batch_size=1000)
eigenmodel.evaluate(val_set2, val_eigenvalue_set)

(40000, 7)
Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 49)                2450      
_________________________________________________________________
dense_29 (Dense)             (None, 50)                2500      
_________________________________________________________________
dense_30 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_31 (Dense)             (None, 7)                 357       
Total params: 7,857
Trainable params: 7,857
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
40000/40000 [==============================] - 0s 7us/step - loss: 0.8205
Epoch 2/20
40000/40000 [==============================] - 0s 4us/step - loss: 0.8158
Epoch 3/20
40000/40000 [==============================] - 0s 4us/ste

0.8561437980651856